In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import LabelEncoder
from scipy.stats import mode
import pandas_profiling
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,KFold
from sklearn.metrics import classification_report,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.metrics import auc

import os
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_columns',5000)

encoder = LabelEncoder()

In [5]:
df = pd.read_csv('final_train.csv')
df.head()

,Unnamed: 0,coupon_id,customer_id,redemption_status,brand,brand_type,category,cd_sum,coupon_discount_x,coupon_used_x,item_counts,no_of_customers,od_sum,other_discount_x,price_sum,qu_sum,quantity_x,selling_price_x,t_counts,total_discount_mean,total_discount_sum,campaign_type,campaign_duration,age_range,marital_status,rented,family_size,no_of_children,income_bracket,coupon_discount_y,coupon_used_y,day,dow,no_of_items,month,other_discount_y,quantity_y,selling_price_y,cdd_sum,customer_id_count,odd_sum,qa_sum,pprice_sum
0,0,27,1053,0,1105,0,6,-5349.93,-1.241694,191.0,125,19.224000,-177447.090167,-44.367382,436274.446738,4661.0,1.169768,117.276864,3713.0,-45.609077,-182797.020167,0,47,3.0,1.0,0.0,1.0,0.0,5.0,-0.287258,1,11,2,208,5,-25.583099,340.487097,163.966826,-89.05,310,-7930.760842,105551,50829.715972
1,1,116,48,0,56,1,6,0.00,0.000000,0.0,3,20.333333,-586.140000,-10.207037,5323.510000,86.0,1.050926,70.885046,75.0,-10.207037,-586.140000,0,47,2.0,0.0,0.0,2.0,0.0,3.0,-3.215039,12,4,5,244,6,-20.013685,31.540260,188.638474,-1237.79,385,-7705.268702,12143,72625.812434
2,2,635,205,0,560,0,11,-605.54,-2.987321,15.0,67,1.761194,-12475.083333,-76.767366,48980.998333,202.0,1.353802,342.826195,142.0,-79.754687,-13080.623333,1,32,3.0,0.0,0.0,2.0,0.0,7.0,-2.212082,85,27,2,533,3,-12.873370,1.392784,112.067666,-2145.72,970,-12487.169143,1351,108705.636349
3,3,644,1050,0,611,0,6,-17.81,-4.452500,1.0,4,3.500000,-260.750000,-4.451964,3697.783333,47.0,1.110390,88.694903,39.0,-8.904464,-278.560000,0,47,3.0,0.0,0.0,2.0,0.0,5.0,-0.751477,2,9,4,216,8,-12.880868,1.291139,100.896997,-178.10,237,-3052.765833,306,23912.588333
4,4,1017,1489,0,1558,0,6,-569.92,-0.198055,23.0,32,27.312500,-46796.046667,-30.665116,289181.848333,1610.0,1.071044,200.991719,1420.0,-30.863170,-47365.966667,0,48,3.0,0.0,0.0,2.0,0.0,3.0,-0.471548,10,2,4,327,5,-12.264174,247.443060,85.016352,-265.01,562,-6892.466021,139063,47779.189609


In [6]:
df.drop(columns=['Unnamed: 0'],inplace=True)
df.head()

,coupon_id,customer_id,redemption_status,brand,brand_type,category,cd_sum,coupon_discount_x,coupon_used_x,item_counts,no_of_customers,od_sum,other_discount_x,price_sum,qu_sum,quantity_x,selling_price_x,t_counts,total_discount_mean,total_discount_sum,campaign_type,campaign_duration,age_range,marital_status,rented,family_size,no_of_children,income_bracket,coupon_discount_y,coupon_used_y,day,dow,no_of_items,month,other_discount_y,quantity_y,selling_price_y,cdd_sum,customer_id_count,odd_sum,qa_sum,pprice_sum
0,27,1053,0,1105,0,6,-5349.93,-1.241694,191.0,125,19.224000,-177447.090167,-44.367382,436274.446738,4661.0,1.169768,117.276864,3713.0,-45.609077,-182797.020167,0,47,3.0,1.0,0.0,1.0,0.0,5.0,-0.287258,1,11,2,208,5,-25.583099,340.487097,163.966826,-89.05,310,-7930.760842,105551,50829.715972
1,116,48,0,56,1,6,0.00,0.000000,0.0,3,20.333333,-586.140000,-10.207037,5323.510000,86.0,1.050926,70.885046,75.0,-10.207037,-586.140000,0,47,2.0,0.0,0.0,2.0,0.0,3.0,-3.215039,12,4,5,244,6,-20.013685,31.540260,188.638474,-1237.79,385,-7705.268702,12143,72625.812434
2,635,205,0,560,0,11,-605.54,-2.987321,15.0,67,1.761194,-12475.083333,-76.767366,48980.998333,202.0,1.353802,342.826195,142.0,-79.754687,-13080.623333,1,32,3.0,0.0,0.0,2.0,0.0,7.0,-2.212082,85,27,2,533,3,-12.873370,1.392784,112.067666,-2145.72,970,-12487.169143,1351,108705.636349
3,644,1050,0,611,0,6,-17.81,-4.452500,1.0,4,3.500000,-260.750000,-4.451964,3697.783333,47.0,1.110390,88.694903,39.0,-8.904464,-278.560000,0,47,3.0,0.0,0.0,2.0,0.0,5.0,-0.751477,2,9,4,216,8,-12.880868,1.291139,100.896997,-178.10,237,-3052.765833,306,23912.588333
4,1017,1489,0,1558,0,6,-569.92,-0.198055,23.0,32,27.312500,-46796.046667,-30.665116,289181.848333,1610.0,1.071044,200.991719,1420.0,-30.863170,-47365.966667,0,48,3.0,0.0,0.0,2.0,0.0,3.0,-0.471548,10,2,4,327,5,-12.264174,247.443060,85.016352,-265.01,562,-6892.466021,139063,47779.189609


In [7]:
x = df.drop('redemption_status',axis=1)
y = df['redemption_status']
col_names = ['cd_sum','coupon_discount_x', 'coupon_used_x', 'item_counts', 'no_of_customers',
       'od_sum', 'other_discount_x', 'price_sum', 'qu_sum', 'quantity_x',
       'selling_price_x', 't_counts', 'total_discount_mean',
       'total_discount_sum', 'campaign_type', 'campaign_duration',
        'family_size', 'no_of_children',
       'income_bracket', 'coupon_discount_y', 'coupon_used_y',
       'no_of_items', 'other_discount_y', 'quantity_y',
       'selling_price_y', 'cdd_sum', 'customer_id_count', 'odd_sum', 'qa_sum',
       'pprice_sum']
features = x[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
x[col_names] = features

In [8]:
cov_matrix = np.cov(x.T)
cov_matrix

array([[ 1.08877597e+05,  1.13816469e+03,  1.33253099e+05, ...,
        -1.06833583e+01,  3.94960293e+00,  1.09482470e+01],
       [ 1.13816469e+03,  2.08676599e+05, -1.07965916e+03, ...,
        -4.91418507e+01,  1.06362908e+01,  4.94098188e+01],
       [ 1.33253099e+05, -1.07965916e+03,  1.77574069e+06, ...,
        -4.41790760e+00,  1.17398602e+01,  1.12328614e+01],
       ...,
       [-1.06833583e+01, -4.91418507e+01, -4.41790760e+00, ...,
         1.00001276e+00, -3.63286111e-01, -8.66594002e-01],
       [ 3.94960293e+00,  1.06362908e+01,  1.17398602e+01, ...,
        -3.63286111e-01,  1.00001276e+00,  4.19143569e-01],
       [ 1.09482470e+01,  4.94098188e+01,  1.12328614e+01, ...,
        -8.66594002e-01,  4.19143569e-01,  1.00001276e+00]])

In [9]:
eigenvalues,eigenvectors = np.linalg.eig(cov_matrix)
eigenvalues,eigenvectors

(array([ 1.78632731e+06,  9.82790809e+04,  2.08689583e+05,  7.27059631e+01,
         6.99538181e+00,  6.55896468e+00,  5.72820533e+00,  4.51460567e+00,
         2.96579647e+00,  2.24058366e+00,  2.26027975e+00,  1.80314051e+00,
         1.62938487e+00,  1.44259621e+00,  1.23595335e+00,  1.03471600e+00,
         9.39826468e-01,  8.92430784e-01,  8.66392924e-01,  7.62060363e-01,
         6.89814661e-01,  5.16520692e-01,  4.79972959e-01,  4.58406683e-01,
         2.52949650e-01,  2.06212936e-01,  2.17733193e-01,  1.21120786e-01,
         9.50144354e-02,  8.86221981e-02,  7.10074737e-02,  4.41423473e-02,
         3.13756663e-02,  2.83752979e-02,  1.11122308e-02,  5.04277861e-03,
         2.21785545e-03,  2.38363627e-03,  3.70993601e-04, -1.77198546e-14,
         2.74919248e-14]),
 array([[ 7.91880628e-02,  9.96796575e-01,  1.10661038e-02, ...,
         -1.37506667e-06, -1.83009591e-17, -3.23731339e-16],
        [-6.25063304e-04, -1.10515310e-02,  9.99938459e-01, ...,
         -1.63917498e-

In [10]:
eig_pairs = [(eigenvalues[i],eigenvectors[:,i]) for i in range(len(eigenvalues))]
eig_pairs

[(1786327.3129779133,
  array([ 7.91880628e-02, -6.25063304e-04,  9.96859316e-01, -1.23851308e-04,
          4.67583937e-04,  7.27726226e-05,  5.92817395e-05, -7.44506354e-05,
         -5.78884672e-05, -2.33316855e-04,  6.83107687e-05,  2.76511969e-05,
         -7.17177560e-05, -7.19037333e-05, -3.18557428e-05,  1.61393018e-04,
         -7.31009945e-05,  4.11929705e-05,  6.88795051e-05,  4.61692913e-05,
          2.86182773e-06, -7.53513836e-06,  8.63619069e-07,  3.73700775e-07,
          6.14221880e-06,  7.67730983e-06,  4.30064984e-06, -2.72151687e-07,
          5.14561452e-06,  2.32001567e-05,  1.07848360e-05,  7.88022157e-06,
          7.81019021e-07,  6.32627331e-06,  3.24696064e-06,  2.75951318e-06,
         -3.88975048e-06,  6.14649587e-06, -2.92185542e-06,  6.72281713e-06,
          6.73659603e-06])),
 (98279.08085024971,
  array([ 9.96796575e-01, -1.10515310e-02, -7.91906206e-02, -7.38060866e-04,
          3.43799088e-04,  2.99766942e-04, -4.91093776e-04, -2.94225707e-04,
    

In [11]:
eig_pairs.sort()
eig_pairs.reverse()
eig_pairs

[(1786327.3129779133,
  array([ 7.91880628e-02, -6.25063304e-04,  9.96859316e-01, -1.23851308e-04,
          4.67583937e-04,  7.27726226e-05,  5.92817395e-05, -7.44506354e-05,
         -5.78884672e-05, -2.33316855e-04,  6.83107687e-05,  2.76511969e-05,
         -7.17177560e-05, -7.19037333e-05, -3.18557428e-05,  1.61393018e-04,
         -7.31009945e-05,  4.11929705e-05,  6.88795051e-05,  4.61692913e-05,
          2.86182773e-06, -7.53513836e-06,  8.63619069e-07,  3.73700775e-07,
          6.14221880e-06,  7.67730983e-06,  4.30064984e-06, -2.72151687e-07,
          5.14561452e-06,  2.32001567e-05,  1.07848360e-05,  7.88022157e-06,
          7.81019021e-07,  6.32627331e-06,  3.24696064e-06,  2.75951318e-06,
         -3.88975048e-06,  6.14649587e-06, -2.92185542e-06,  6.72281713e-06,
          6.73659603e-06])),
 (208689.58331971915,
  array([ 1.10661038e-02,  9.99938459e-01, -2.52112142e-04, -5.98273689e-06,
          6.95029290e-05,  6.63077317e-06, -1.84913616e-05, -6.53698300e-06,
   

In [12]:
eigenvale_sort = [eig_pairs[i][0] for i in range(len(eigenvalues))]
eigenvector_sort = [eig_pairs[i][1] for i in range(len(eigenvalues))]

In [13]:
tot = sum(eigenvalues)
var_exp = [(i/tot)*100 for i in sorted(eigenvalues,reverse=True)]
cum_var_exp = np.cumsum(var_exp)
cum_var_exp

array([ 85.33082414,  95.2996882 ,  99.99436811,  99.9978412 ,
        99.99817536,  99.99848867,  99.9987623 ,  99.99897796,
        99.99911963,  99.9992276 ,  99.99933463,  99.99942077,
        99.9994986 ,  99.99956751,  99.99962655,  99.99967598,
        99.99972087,  99.9997635 ,  99.99980489,  99.99984129,
        99.99987424,  99.99989892,  99.99992185,  99.99994374,
        99.99995583,  99.99996623,  99.99997608,  99.99998186,
        99.9999864 ,  99.99999064,  99.99999403,  99.99999614,
        99.99999764,  99.99999899,  99.99999952,  99.99999976,
        99.99999988,  99.99999998, 100.        , 100.        ,
       100.        ])

In [14]:
p_reduce = np.array(eigenvector_sort[0:5]).transpose()
p_reduce

array([[ 7.91880628e-02,  1.10661038e-02,  9.96796575e-01,
         1.73194003e-05,  1.11358124e-03],
       [-6.25063304e-04,  9.99938459e-01, -1.10515310e-02,
        -4.88355029e-04, -4.87933107e-06],
       [ 9.96859316e-01, -2.52112142e-04, -7.91906206e-02,
         2.18713716e-05,  5.21250875e-05],
       [-1.23851308e-04, -5.98273689e-06, -7.38060866e-04,
        -1.97087394e-04, -3.24073868e-02],
       [ 4.67583937e-04,  6.95029290e-05,  3.43799088e-04,
        -8.20935927e-04,  9.51447174e-02],
       [ 7.27726226e-05,  6.63077317e-06,  2.99766942e-04,
        -5.88098630e-04, -3.46095490e-01],
       [ 5.92817395e-05, -1.84913616e-05, -4.91093776e-04,
         2.46775439e-04, -2.94878057e-03],
       [-7.44506354e-05, -6.53698300e-06, -2.94225707e-04,
         4.94124523e-04,  3.54359515e-01],
       [-5.78884672e-05, -1.03211455e-05, -4.45796357e-04,
         6.55353393e-04,  3.18373253e-01],
       [-2.33316855e-04, -2.14430250e-05, -4.83290174e-04,
        -9.55738862e-05

In [15]:
pc_data = np.dot(x,p_reduce)
pc_data.shape

(78369, 5)

In [16]:
pc_data

array([[ 1.10301206e+03,  1.05294956e+03, -7.22293797e+01,
        -1.15178886e+01,  1.69540638e+00],
       [ 6.49827146e+01,  4.92639550e+01,  1.10662980e+02,
        -4.00778319e+00,  1.09824160e-01],
       [ 6.08403690e+02,  2.11861123e+02,  5.86361862e+02,
        -2.70726346e+01,  1.60592064e+00],
       ...,
       [ 9.20369589e+02,  1.01205924e+03,  6.72070595e+02,
        -1.24497216e+01,  2.30176026e+00],
       [ 2.15352875e+01,  7.24606475e+01,  1.31914449e+02,
        -2.99870795e+01,  1.90408831e-01],
       [ 7.37386183e+02,  6.30319897e+02,  6.17609167e+02,
        -8.28016813e+00,  1.17306649e+00]])

In [17]:
x_train,x_test,y_train,y_test = train_test_split(pc_data,y,test_size=0.3,random_state=1990)

In [18]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((54858, 5), (54858,), (23511, 5), (23511,))

# Logistic Regression

In [19]:
LR = LogisticRegression()
LR.fit(x_train,y_train)
y_pred_LR = LR.predict(x_test)
print(classification_report(y_test,y_pred_LR))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     23278
           1       0.00      0.00      0.00       233

    accuracy                           0.99     23511
   macro avg       0.50      0.50      0.50     23511
weighted avg       0.98      0.99      0.99     23511



In [20]:
roc_auc_score(y_test,y_pred_LR)
Model = ['Logistic Regression']
ROC_AUC_Accuracy = [roc_auc_score(y_test,y_pred_LR)]

# kNN

## Caution --- Algorithm - 'brute' was crashing the system, hence removed it.

In [21]:
params = {
    
    'n_neighbors': range(1,5),
    'weights': ['uniform','distance'],
    'algorithm': ['ball_tree','kd_tree'],
    'p': [1,2,3]
}

knn = KNeighborsClassifier()

gs = GridSearchCV(estimator=knn,n_jobs=-1,cv=3,param_grid=params,scoring='recall')
gs.fit(pc_data,y)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid={'algorithm': ['ball_tree', 'kd_tree'],
                         'n_neighbors': range(1, 5), 'p': [1, 2, 3],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='recall', verbose=0)

In [22]:
knn = KNeighborsClassifier(**gs.best_params_)
knn.fit(x_train,y_train)
y_pred_knn = knn.predict(x_test)
print(roc_auc_score(y_test,y_pred_knn))
Model.append('k-Nearest-Neighbours')
ROC_AUC_Accuracy.append(roc_auc_score(y_test,y_pred_knn))

0.5343736667493889


# Naive Bayes

In [23]:
nb = GaussianNB()
nb.fit(x_train,y_train)
y_pred_nb = nb.predict(x_test)
print(roc_auc_score(y_test,y_pred_nb))
Model.append('Naive Bayes')
ROC_AUC_Accuracy.append(roc_auc_score(y_test,y_pred_nb))

0.5359826017824489


In [24]:
print(classification_report(y_test,y_pred_nb))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     23278
           1       0.08      0.08      0.08       233

    accuracy                           0.98     23511
   macro avg       0.53      0.54      0.54     23511
weighted avg       0.98      0.98      0.98     23511



# Decision Tree

In [25]:
params = {
    
    'criterion':['gini','entropy'],
    'splitter':['best','random'],
    'max_depth':range(1,10),
    'max_leaf_nodes':range(2,10,1),
    'max_features':['auto','log2']
    
}

dt = DecisionTreeClassifier()

gs = GridSearchCV(estimator=dt,n_jobs=-1,cv=3,param_grid=params,scoring='recall')
gs.fit(pc_data,y)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 10),
                         'm

In [26]:
dt = DecisionTreeClassifier(**gs.best_params_)
dt.fit(x_train,y_train)
y_pred_dt = dt.predict(x_test)
print(roc_auc_score(y_test,y_pred_dt))
Model.append('Decision Tree')
ROC_AUC_Accuracy.append(roc_auc_score(y_test,y_pred_dt))

0.5


In [27]:
print(classification_report(y_test,y_pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     23278
           1       0.00      0.00      0.00       233

    accuracy                           0.99     23511
   macro avg       0.50      0.50      0.50     23511
weighted avg       0.98      0.99      0.99     23511



In [28]:
Model,ROC_AUC_Accuracy

(['Logistic Regression',
  'k-Nearest-Neighbours',
  'Naive Bayes',
  'Decision Tree'],
 [0.5, 0.5343736667493889, 0.5359826017824489, 0.5])

# Random Forest

In [29]:
params = {
    
    'n_estimators':range(10,100,10),
    'criterion':['gini','entropy'],
    'max_depth':range(2,10,1),
    'max_leaf_nodes':range(2,10,1),
    'max_features':['auto','log2']
    
}

rf = RandomForestClassifier()

rs = RandomizedSearchCV(estimator=rf,param_distributions=params,cv=3,scoring='recall',n_jobs=-1)
rs.fit(pc_data,y)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [30]:
rf = RandomForestClassifier(**rs.best_params_)
rf.fit(x_train,y_train)
y_pred_rf = rf.predict(x_test)
print(roc_auc_score(y_test,y_pred_rf))
Model.append('Random Forest')
ROC_AUC_Accuracy.append(roc_auc_score(y_test,y_pred_rf))

0.5


In [31]:
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     23278
           1       0.00      0.00      0.00       233

    accuracy                           0.99     23511
   macro avg       0.50      0.50      0.50     23511
weighted avg       0.98      0.99      0.99     23511



# Now using the Bagging Classifiers

In [32]:
LR_Bag = BaggingClassifier(base_estimator=LR,n_estimators=100,n_jobs=-1,random_state=1)
knn_Bag = BaggingClassifier(base_estimator=knn,n_estimators=100,n_jobs=-1,random_state=1)
nb_Bag = BaggingClassifier(base_estimator=nb,n_estimators=100,n_jobs=-1,random_state=1)
dt_Bag = BaggingClassifier(base_estimator=dt,n_estimators=100,n_jobs=-1,random_state=1)

In [33]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
for model,name in zip([LR_Bag,knn_Bag,nb_Bag,dt_Bag],['Bagged-LR','Bagged-kNN','Bagged-NB','Bagged-DT']):
    roc_acc = []
    for train,test in kf.split(pc_data,y):
        x_train = pc_data[train,:]
        x_test = pc_data[test,:]
        y_train = y[train]
        y_test = y[test]
        model.fit(x_train,y_train)
        y_pred = model.predict(x_test)
        score = roc_auc_score(y_test,y_pred)
        fpr,tpr,_ = roc_curve(y_test,y_pred)
        roc_acc.append(auc(fpr,tpr))
    Model.append(name)
    ROC_AUC_Accuracy.append(np.mean(roc_acc))
    print('The AUC Score for')
    print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))
    print()

The AUC Score for
Bagged-LR is 0.50 with variacne of (+/-) 0.00000

The AUC Score for
Bagged-kNN is 0.55 with variacne of (+/-) 0.00011

The AUC Score for
Bagged-NB is 0.54 with variacne of (+/-) 0.00003

The AUC Score for
Bagged-DT is 0.50 with variacne of (+/-) 0.00000



## Now Moving Towards Boosting

In [34]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMModel,LGBMClassifier

In [35]:
LR_Boost = AdaBoostClassifier(base_estimator=LR,n_estimators=100,learning_rate=0.01,random_state=1)
knn_Boost = AdaBoostClassifier(base_estimator=knn,n_estimators=100,learning_rate=0.01,random_state=1)
nb_Boost = AdaBoostClassifier(base_estimator=nb,n_estimators=100,learning_rate=0.01,random_state=1)
dt_Boost = AdaBoostClassifier(base_estimator=dt,n_estimators=100,learning_rate=0.01,random_state=1)
rf_Boost = AdaBoostClassifier(base_estimator=rf,n_estimators=100,learning_rate=0.01,random_state=1)
gb_Boost = GradientBoostingClassifier(n_estimators=100,learning_rate=0.01)
lgbm = LGBMClassifier(objective='binary',n_estimators=100,reg_alpha=2,reg_lambda=5,random_state=1,learning_rate=0.01,is_unbalance=True)

In [36]:
from sklearn.metrics import auc
kf = KFold(n_splits=3,shuffle=True,random_state=1)
for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
    roc_acc = []
    for train,test in kf.split(pc_data,y):
        x_train = pc_data[train,:]
        x_test = pc_data[test,:]
        y_train = y[train]
        y_test = y[test]
        model.fit(x_train,y_train)
        y_pred = model.predict(x_test)
        score = roc_auc_score(y_test,y_pred)
        fpr,tpr,_ = roc_curve(y_test,y_pred)
        roc_acc.append(auc(fpr,tpr))
    Model.append(name)
    ROC_AUC_Accuracy.append(np.mean(roc_acc))
    print('The AUC Score for')
    print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))
    print()

The AUC Score for
Boosted-LR is 0.50 with variacne of (+/-) 0.00000

The AUC Score for
Boosted-NB is 0.51 with variacne of (+/-) 0.00002

The AUC Score for
Boosted-DT is 0.50 with variacne of (+/-) 0.00000

The AUC Score for
Boosted - Random Forest is 0.50 with variacne of (+/-) 0.00000

The AUC Score for
LGBM is 0.62 with variacne of (+/-) 0.00006



In [37]:
final_result = pd.DataFrame({'Model - PCA Data':Model,'Accuracy':ROC_AUC_Accuracy})
final_result

,Model - PCA Data,Accuracy
0,Logistic Regression,0.500000
1,k-Nearest-Neighbours,0.534374
2,Naive Bayes,0.535983
3,Decision Tree,0.500000
4,Random Forest,0.500000
5,Bagged-LR,0.500000
6,Bagged-kNN,0.545510
7,Bagged-NB,0.540775
8,Bagged-DT,0.500000
9,Boosted-LR,0.499974
